In [1]:
import os

In [2]:
%pwd

'e:\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
     config_filepath=CONFIG_FILE_PATH,
     params_filepath=PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)    

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config= self.config.model_trainer
        params= self.params.TrainingArguments

        create_directories([config.root_dir])
        model_trainer_config= ModelTrainerConfig(
    root_dir= config.root_dir,
    data_path= config.data_path,
    model_ckpt= config.model_ckpt,
    num_train_epochs= params.num_train_epochs,
    warmup_steps= params.warmup_steps,
    per_device_train_batch_size= params.per_device_train_batch_size ,
    weight_decay= params.weight_decay,
    logging_steps= params.logging_steps,
    eval_strategy= params.eval_strategy,
    eval_steps= params.eval_steps,
    save_steps= params.save_steps,
    gradient_accumulation_steps= params.gradient_accumulation_steps
        )
        return model_trainer_config

In [8]:

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


c:\Users\dell\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-07 15:13:55,053: INFO: config: PyTorch version 2.4.1 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        dataset_samsum_pt = load_from_disk(self.config.data_path)        
        small_train=dataset_samsum_pt["train"].select(range(5))
        small_test=dataset_samsum_pt["validation"].select(range(2))
        

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.eval_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=small_train, 
                  eval_dataset=small_test)
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config=ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
    
except Exception as e:
    raise e

[2025-07-07 15:13:55,666: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-07 15:13:55,672: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-07 15:13:55,674: INFO: common: Created dictionary at:{path}]
[2025-07-07 15:13:55,676: INFO: common: Created dictionary at:{path}]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\dell\anaconda3\envs\textS\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\dell\AppData\Local\Temp\ipykernel_21876\187370102.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,
                                              
100%|██████████| 1/1 [02:49<00:00, 161.04s/it]c:\Users\dell\anaconda3\envs\textS\lib\site-packages\transfor

{'eval_loss': 3.1115856170654297, 'eval_runtime': 7.4847, 'eval_samples_per_second': 0.267, 'eval_steps_per_second': 0.267, 'epoch': 1.0}


100%|██████████| 1/1 [04:06<00:00, 246.70s/it]


{'train_runtime': 246.5192, 'train_samples_per_second': 0.02, 'train_steps_per_second': 0.004, 'train_loss': 1.448084831237793, 'epoch': 1.0}
